In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('basics').getOrCreate()

In [2]:
# Let's read in the data. Note that it's in the format of JSON.
df = spark.read.options(header="True", inferSchema="True").csv('database.csv')

In [3]:
#df.count()
#df.columns
#df.dtypes

In [4]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType,BooleanType)

In [5]:
data_schema = [StructField('Engine Cylinders',IntegerType(),True),
              StructField('Engine Displacement',IntegerType(),True),
              StructField('Turbocharger',StringType(),True),
              StructField('Supercharger',StringType(),True),
              StructField('Vehicle ID',IntegerType(),True),
              StructField('Year',IntegerType(),True),
              StructField('Make',StringType(),True),
              StructField('Drive',StringType(),True),
              StructField('Model',StringType(),True),
              StructField('Class',StringType(),True),
              StructField('Transmission',StringType(),True),
              StructField('Fuel Type',StringType(),True),
              StructField('Fuel Economy Score',IntegerType(),True),
              StructField('Gasoline/Electricity Blended (CD)',BooleanType(),True)]
final_struct = StructType(fields=data_schema)


In [6]:
from pyspark.sql.functions import mean, stddev, min, max

mean = df.select(mean(df['City MPG'])).collect()
mean_val = mean[0][0]
stddev = df.select(stddev(df['City MPG'])).collect()
stddev_val = stddev[0][0]
minimum=df.select(min(df['City MPG'])).collect()
maximum=df.select(max(df['City MPG'])).collect()
minimum=minimum[0][0]
maximum=maximum[0][0]
max_range = mean_val + 3*stddev_val
min_range = mean_val - 3*stddev_val

#min_range, max_range, minimum, maximum

In [7]:
df = df.filter(df['City MPG']<max_range)

maximum=df.select(max(df['City MPG'])).collect()
maximum=maximum[0][0]

#df.count(), maximum


In [8]:
df = spark.read.options(header="True", inferSchema="True").csv('database.csv',schema=final_struct)
df = df.filter(df['Fuel Economy Score']>0)
#df.count()
#df.printSchema()

In [9]:
df.fillna('N', subset=['Supercharger','Turbocharger'])
df.na.drop()
#df.count()

DataFrame[Engine Cylinders: int, Engine Displacement: int, Turbocharger: string, Supercharger: string, Vehicle ID: int, Year: int, Make: string, Drive: string, Model: string, Class: string, Transmission: string, Fuel Type: string, Fuel Economy Score: int, Gasoline/Electricity Blended (CD): boolean]

In [11]:
from pyspark.sql.functions import floor
from pyspark.sql import Row
df=df.withColumn("bin_score", floor(df['Fuel Economy Score']/4 + 1))
#df.printSchema()


In [13]:
#df.toPandas().to_csv('clean data.csv')